<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/black.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Upload images** { display-mode: "form" }

import shutil
from tqdm import tqdm
import os
import shutil, sys
import re
import io
import IPython.display
import numpy as np
import PIL.Image
from google.colab import files
import shutil

upload_folder = "/content/upload"
result_folder = "/content/results"

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder)

if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

basepath = os.getcwd()
uploaded = files.upload()
for filename in uploaded.keys():
    shutil.move(os.path.join(basepath, filename), os.path.join(upload_folder, filename))

In [ ]:
#@title ##**Enhance Blacks** { display-mode: "form" }
import cv2
import numpy as np
import PIL.Image
from IPython.display import display, clear_output
import ipywidgets as widgets
import os
import shutil

upload_folder = "/content/upload"
result_folder = "/content/results"

if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)
if not os.path.isdir(upload_folder):
    os.makedirs(upload_folder)

def enhance_blacks(image, black_level=0.5, threshold=0.1, transition=0.05, curve_strength=0.5):
    image_float = image.astype(np.float32) / 255.0
    corrected = np.copy(image_float)
    threshold = max(threshold, 1e-6)
    mask = image_float < threshold
    linear_factor = (1 - black_level)
    nonlinear_factor = np.zeros_like(image_float[mask])
    if threshold > 1e-6:
        nonlinear_factor = np.power(np.clip(image_float[mask] / threshold, 0, None), black_level * curve_strength)
    corrected[mask] = image_float[mask] * (
        (1 - curve_strength) * linear_factor + curve_strength * nonlinear_factor
    )
    transition_start = threshold - transition
    transition_end = threshold
    transition_mask = (image_float >= transition_start) & (image_float < transition_end)
    for channel in range(3):
        alpha = np.clip((image_float[:, :, channel] - transition_start) / max(transition, 1e-6), 0, 1)
        nonlinear_component = np.zeros_like(image_float[:, :, channel])
        if threshold > 1e-6:
            base = np.clip(image_float[:, :, channel] / threshold, 0, None)
            nonlinear_component = np.power(base, black_level * curve_strength)
        corrected[:, :, channel][transition_mask[:, :, channel]] = (
            (1 - alpha) * (
                image_float[:, :, channel] * (
                    (1 - curve_strength) * linear_factor +
                    curve_strength * nonlinear_component
                )
            ) +
            alpha * image_float[:, :, channel]
        )[transition_mask[:, :, channel]]
    corrected = np.clip(corrected, 0, 1) * 255
    return corrected.astype(np.uint8)

def create_combined_image(image, black_level, threshold, transition, curve_strength, blend_factor):
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
    enhanced_image = enhance_blacks(image, black_level, threshold, transition, curve_strength)
    enhanced_rgb = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB).astype(np.float32)
    final_rgb = (1 - blend_factor) * img_rgb + blend_factor * enhanced_rgb
    image_original = PIL.Image.fromarray(img_rgb.astype(np.uint8))
    image_enhanced = PIL.Image.fromarray(final_rgb.clip(0, 255).astype(np.uint8))
    max_width = 500
    width_original, height_original = image_original.size
    width_enhanced, height_enhanced = image_enhanced.size
    if width_original > max_width:
        new_height = int(height_original * max_width / width_original)
        image_original = image_original.resize((max_width, new_height))
    if width_enhanced > max_width:
        new_height = int(height_enhanced * max_width / width_enhanced)
        image_enhanced = image_enhanced.resize((max_width, new_height))
    combined_image = PIL.Image.fromarray(np.hstack((np.array(image_original), np.array(image_enhanced))))
    return combined_image

def update_image(black_level, threshold, transition, curve_strength, blend_factor):
    global preview_image
    if preview_image is None:
        with output:
            clear_output(wait=True)
            print("No image available for preview.")
        return
    combined_image = create_combined_image(preview_image, black_level, threshold, transition, curve_strength, blend_factor)
    with output:
        clear_output(wait=True)
        display(combined_image)

filenames = os.listdir(upload_folder)
preview_image = None
if filenames:
    preview_path = os.path.join(upload_folder, filenames[0])
    preview_image = cv2.imread(preview_path)
    if preview_image is None:
        print(f"Failed to load image: {preview_path}")

slider_black_level = widgets.FloatSlider(value=0.9, min=0.0, max=1.0, step=0.1, description='Black Level')
slider_threshold = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.05, description='Threshold')
slider_transition = widgets.FloatSlider(value=0.1, min=0.0, max=0.2, step=0.01, description='Transition')
slider_curve_strength = widgets.FloatSlider(value=0.9, min=0.0, max=1.0, step=0.1, description='Curve Strength')
slider_blend = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Blend Factor')

display(widgets.VBox([slider_black_level, slider_threshold, slider_transition, slider_curve_strength, slider_blend]))

output = widgets.Output()
display(output)

def on_value_change(change):
    update_image(
        slider_black_level.value,
        slider_threshold.value,
        slider_transition.value,
        slider_curve_strength.value,
        slider_blend.value
    )

slider_black_level.observe(on_value_change, names='value')
slider_threshold.observe(on_value_change, names='value')
slider_transition.observe(on_value_change, names='value')
slider_curve_strength.observe(on_value_change, names='value')
slider_blend.observe(on_value_change, names='value')

on_value_change(None)

In [ ]:
#@title ##**Download** { display-mode: "form" }
from google.colab import files
import cv2
import numpy as np
from IPython.display import display

black_level = slider_black_level.value
threshold = slider_threshold.value
transition = slider_transition.value
curve_strength = slider_curve_strength.value
blend_factor = slider_blend.value

img_rgb = cv2.cvtColor(preview_image, cv2.COLOR_BGR2RGB).astype(np.float32)
enhanced_image = enhance_blacks(preview_image, black_level, threshold, transition, curve_strength)
enhanced_rgb = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB).astype(np.float32)
final_rgb = (1 - blend_factor) * img_rgb + blend_factor * enhanced_rgb
final_image = final_rgb.clip(0, 255).astype(np.uint8)
final_image_bgr = cv2.cvtColor(final_image, cv2.COLOR_RGB2BGR)

output_filename = 'processed_image.jpg'
cv2.imwrite(output_filename, final_image_bgr)
files.download(output_filename)

print(f"Image saved as {output_filename} and downloaded")